<a href="https://colab.research.google.com/github/irohan98/Agents-Practice/blob/main/AI_learnings.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Quickstart Guide

https://google.github.io/adk-docs/get-started/quickstart/#run-your-agent

### Workflow


The agent takes the user’s request, figures out how best to respond (possibly using tools, memory, or even other agents), and calls an LLM (with secure credentials) to generate responses. So the LLM serves the agent — not the other way around.

```
[ USER ]
   │
   ▼
┌─────────────────────┐
│     Agent Layer     │   <-- Your agent (logic, memory, tools)
└─────────────────────┘
   │
┌──┴───────────┬────────────┐
│              │            │
▼              ▼            ▼
[ Tools ]   [ Memory ]   [ Other Agents ]
   │
   ▼
┌────────────────────────────┐
│   LLM API Request Layer    │   <-- Authenticated API call
└────────────────────────────┘
   │
   ▼
┌────────────────────────────┐
│     External LLM Service   │   <-- OpenAI GPT, Anthropic Claude, etc.
└────────────────────────────┘
   │
   ▼
[ Response Output ]
   │
   ▼
[ USER ]
```

#### Example Use Case Flow

1. You ask: "Summarize this research paper."

2. Agent:
  
  a. Stores your history in memory.
  
  b. Parses your input.

  c. Might retrieve context via tools (e.g., web search).

3. Sends a secure API call to the LLM to generate the summary.

4. LLM returns a response.

5. Agent wraps it up (maybe formats or verifies it) and sends it to you.




In [1]:
# Install packages here or use terminal

!pip install google-adk -q
!pip install timezonefinder
!pip install pyngrok
!pip install python-dotenv

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.1/40.1 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 55.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 240.0/240.0 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 218.1/218.1 kB 16.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 335.7/335.7 kB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 150.9/150.9 kB 12.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 65.8/65.8 kB 4.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 118.5/118.5 kB 9.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 196.2/196.2 kB 13.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.8/64.8 kB 4.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.5/229.5 kB 17.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 103.8/103.8 kB 7.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
!mkdir multi_tool_agent

In [3]:
!echo "from . import agent" > multi_tool_agent/__init__.py

In [4]:
!touch multi_tool_agent/agent.py

Paste this code into agent.py (This code is from the tutorial link attached above)

```
import datetime
from zoneinfo import ZoneInfo
from google.adk.agents import Agent

def get_weather(city: str) -> dict:
    """Retrieves the current weather report for a specified city.

    Args:
        city (str): The name of the city for which to retrieve the weather report.

    Returns:
        dict: status and result or error msg.
    """
    if city.lower() == "new york":
        return {
            "status": "success",
            "report": (
                "The weather in New York is sunny with a temperature of 25 degrees"
                " Celsius (77 degrees Fahrenheit)."
            ),
        }
    else:
        return {
            "status": "error",
            "error_message": f"Weather information for '{city}' is not available.",
        }


def get_current_time(city: str) -> dict:
    """Returns the current time in a specified city.

    Args:
        city (str): The name of the city for which to retrieve the current time.

    Returns:
        dict: status and result or error msg.
    """

    if city.lower() == "new york":
        tz_identifier = "America/New_York"
    else:
        return {
            "status": "error",
            "error_message": (
                f"Sorry, I don't have timezone information for {city}."
            ),
        }

    tz = ZoneInfo(tz_identifier)
    now = datetime.datetime.now(tz)
    report = (
        f'The current time in {city} is {now.strftime("%Y-%m-%d %H:%M:%S %Z%z")}'
    )
    return {"status": "success", "report": report}


root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description=(
        "Agent to answer questions about the time and weather in a city."
    ),
    instruction=(
        "You are a helpful agent who can answer user questions about the time and weather in a city."
    ),
    tools=[get_weather, get_current_time],
)
```

In [11]:
# !touch multi_tool_agent/.env

import getpass

KEY = getpass.getpass("Enter your Google API key: ")

with open("multi_tool_agent/.env", "w") as f:
    f.write("GOOGLE_GENAI_USE_VERTEXAI=FALSE\n")
    f.write(f"GOOGLE_API_KEY='{KEY}'\n")

Enter your Google API key: ··········


In [12]:
# Run in Terminal
!adk run multi_tool_agent

INFO:google_adk.google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/publishers\/google\/models\/gemini.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'

In [30]:
# Run using Web App via ngrok

import getpass
token = getpass.getpass("Enter your ngrok token: ")
ngrok.set_auth_token(token)

Enter your ngrok token: ··········


In [31]:
import subprocess

# Run adk web in the background
server = subprocess.Popen(["adk", "web", "--no-reload"])

from pyngrok import ngrok

public_url = ngrok.connect(8000)
print("Public URL:", public_url)

Public URL: NgrokTunnel: "https://5805-34-147-40-12.ngrok-free.app" -> "http://localhost:8000"


In [32]:
server.terminate()
ngrok.kill()

## ----------------------------

## Quickstart (Modified for any city/state)

Steps (by RI):
1. Create a free account on https://home.openweathermap.org/

2. Create an API key on https://home.openweathermap.org/api_keys

3. Include it in the code below for the API call to be successful on your free account

4. Test the code locally and then replace the weather and time functions in the agent.py file

5. Run the agent and interact with it



In [56]:
# Testing the base logic which will be inside agent.py

import datetime
import requests
from zoneinfo import ZoneInfo
from timezonefinder import TimezoneFinder
import getpass

# Add your OpenWeatherMap API Key here

KEY = getpass.getpass("Enter your OpenWeatherMap API Key: ")

API_KEY = KEY

def get_coordinates(city: str) -> tuple:
    """Fetch latitude and longitude for a given city."""
    response = requests.get(
        f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={API_KEY}"
    )
    data = response.json()
    if not data:
        return None  # Return None if no location data is found
    print(f"Data Found for {city}:", data[0]["lat"], data[0]["lon"])
    return data[0]["lat"], data[0]["lon"]

def get_weather(city: str) -> dict:
    coords = get_coordinates(city)
    if not coords:
        return {
            "status": "error",
            "error_message": f"Could not find location for '{city}'."
        }

    lat, lon = coords
    response = requests.get(
        f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    )
    data = response.json()

    if response.status_code != 200:
        return {
            "status": "error",
            "error_message": data.get("message", "Unknown error"),
        }

    weather_desc = data["weather"][0]["description"]
    temp_c = data["main"]["temp"]
    temp_f = (temp_c * 9/5) + 32
    return {
        "status": "success",
        "report": (
            f"The weather in {city} is {weather_desc} with a temperature of "
            f"{temp_c:.1f}°C ({temp_f:.1f}°F)."
        ),
    }

def get_current_time(city: str) -> dict:
    coords = get_coordinates(city)
    if not coords:
        return {
            "status": "error",
            "error_message": f"Could not find location for '{city}'."
        }

    lat, lon = coords
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lat=lat, lng=lon)
    if not tz_name:
        return {
            "status": "error",
            "error_message": f"Could not determine timezone for '{city}'."
        }

    tz = ZoneInfo(tz_name)
    now = datetime.datetime.now(tz)
    report = f"The current time in {city} is {now.strftime('%Y-%m-%d %H:%M:%S %Z%z')}"
    return {"status": "success", "report": report}

In [57]:
print(get_weather("Hyderabad"))

Data Found for Hyderabad: 17.360589 78.4740613
{'status': 'error', 'error_message': 'Invalid API key. Please see https://openweathermap.org/faq#error401 for more info.'}


In [ ]:
import getpass

OPENWEATHERMAP_KEY = getpass.getpass("Enter your OPENWEATHERMAP API Key: ")

with open("multi_tool_agent/.env", "w") as f:
    f.write(f"OPENWEATHERMAP_API_KEY='{OPENWEATHERMAP_KEY}'\n")

In [ ]:
# # [WORKING CODE] IN AGENT.PY FILE

import os
import datetime
import requests
from zoneinfo import ZoneInfo
from timezonefinder import TimezoneFinder
from dotenv import load_dotenv

# Load environment variables from .env
load_dotenv()

# Get the API key from the environment
API_KEY = os.getenv("OPENWEATHERMAP_API_KEY")

if not API_KEY:
    raise ValueError("OPENWEATHERMAP_API_KEY not set in environment or .env file.")

def get_coordinates(city: str) -> tuple:
    response = requests.get(
        f"http://api.openweathermap.org/geo/1.0/direct?q={city}&limit=1&appid={API_KEY}"
    )
    data = response.json()
    if not data:
        return None
    print(f"Data Found for {city}:", data[0]["lat"], data[0]["lon"])
    return data[0]["lat"], data[0]["lon"]

def get_weather(city: str) -> dict:
    coords = get_coordinates(city)
    if not coords:
        return {"status": "error", "error_message": f"Could not find location for '{city}'."}

    lat, lon = coords
    response = requests.get(
        f"https://api.openweathermap.org/data/2.5/weather?lat={lat}&lon={lon}&appid={API_KEY}&units=metric"
    )
    data = response.json()

    if response.status_code != 200:
        return {"status": "error", "error_message": data.get("message", "Unknown error")}

    weather_desc = data["weather"][0]["description"]
    temp_c = data["main"]["temp"]
    temp_f = (temp_c * 9 / 5) + 32
    return {
        "status": "success",
        "report": (
            f"The weather in {city} is {weather_desc} with a temperature of "
            f"{temp_c:.1f}°C ({temp_f:.1f}°F)."
        ),
    }

def get_current_time(city: str) -> dict:
    coords = get_coordinates(city)
    if not coords:
        return {"status": "error", "error_message": f"Could not find location for '{city}'."}

    lat, lon = coords
    tf = TimezoneFinder()
    tz_name = tf.timezone_at(lat=lat, lng=lon)
    if not tz_name:
        return {"status": "error", "error_message": f"Could not determine timezone for '{city}'."}

    tz = ZoneInfo(tz_name)
    now = datetime.datetime.now(tz)
    report = f"The current time in {city} is {now.strftime('%Y-%m-%d %H:%M:%S %Z%z')}"
    return {"status": "success", "report": report}

from google.adk.agents import Agent

root_agent = Agent(
    name="weather_time_agent",
    model="gemini-2.0-flash",
    description="Agent to answer questions about the time and weather in a city.",
    instruction="You are a helpful agent who can answer user questions about the time and weather in a city.",
    tools=[get_weather, get_current_time],
)


In [58]:
# Run in Terminal
!adk run multi_tool_agent

INFO:google_adk.google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/publishers\/google\/models\/gemini.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for gemini-.* from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'>
INFO:google_adk.google.adk.models.registry:Updating LLM class for projects\/.+\/locations\/.+\/endpoints\/.+ from <class 'google.adk.models.google_llm.Gemini'> to <class 'google.adk.models.google_llm.Gemini'

In [59]:
import getpass
token = getpass.getpass("Enter your ngrok token: ")
ngrok.set_auth_token(token)

Enter your ngrok token: ··········


In [60]:
import subprocess

# Run adk web in the background
server = subprocess.Popen(["adk", "web", "--no-reload"])

from pyngrok import ngrok

public_url = ngrok.connect(8000)
print("Public URL:", public_url)


Public URL: NgrokTunnel: "https://bdd1-34-147-40-12.ngrok-free.app" -> "http://localhost:8000"


In [61]:
server.terminate()
ngrok.kill()